In [1]:
import pandas as pd
import ast

In [2]:
file = '/Users/abolfazlansari/Echoes_AI/combined_labels.csv'
df = pd.read_csv(file)

In [ ]:
print(df['paragraph_brunet_index'].mean())
df.groupby('author')['paragraph_brunet_index'].mean()

In [ ]:
grouped = df.groupby(['author', 'category'])['paragraph_brunet_index'].mean()
print(grouped)

In [ ]:
print(df['paragraph_readability_score'].mean())
df.groupby('author')['paragraph_readability_score'].mean()
# AI text are more readable !

In [ ]:
grouped = df.groupby(['author', 'category'])['paragraph_readability_score'].mean()
print(grouped)
# The AI help all 3 category to frame more readable articles, the major effect was on local, then major and colleges together

In [ ]:

print(df['paragraph_pos_ADJ'].mean())
df.groupby('author')['paragraph_pos_ADJ'].mean()

In [ ]:
grouped = df.groupby(['author', 'category'])['paragraph_pos_ADJ'].mean()
print(grouped)


In [ ]:
print(df['paragraph_pos_NUM'].mean())
df.groupby('author')['paragraph_pos_NUM'].mean()

In [ ]:
grouped = df.groupby(['author', 'category'])['paragraph_pos_NUM'].mean()
print(grouped)


In [ ]:
df['paragraph_polarity_scores'] = df['paragraph_polarity_scores'].apply(ast.literal_eval)
df['avg_paragraph_polarity'] = df['paragraph_polarity_scores'].apply(lambda x: sum(x)/len(x) if x else float('nan'))
grouped = df.groupby(['author', 'category'])['avg_paragraph_polarity'].mean()

print(grouped)

# No significant difference

In [ ]:
df['paragraph_subjectivity_scores'] = df['paragraph_subjectivity_scores'].apply(ast.literal_eval)
df['avg_paragraph_subjectivity'] = df['paragraph_subjectivity_scores'].apply(lambda x: sum(x)/len(x) if x else float('nan'))
grouped = df.groupby(['author', 'category'])['avg_paragraph_subjectivity'].mean()

print(grouped)


In [ ]:
df['paragraph_formality_score'] = df['paragraph_formality_score'].apply(ast.literal_eval)
df['avg_paragraph_formality'] = df['paragraph_formality_score'].apply(lambda x: sum(x)/len(x) if x else float('nan'))
grouped = df.groupby(['author', 'category'])['avg_paragraph_formality'].mean()

print(grouped)
# AI make it all less formal


In [ ]:
print(df['paragraph_pos_wc'].mean())
df.groupby('author')['paragraph_pos_wc'].mean()


In [ ]:
grouped = df.groupby(['author', 'category'])['paragraph_pos_wc'].mean()
print(grouped)


In [ ]:
print(df['paragraph_neg_wc'].mean())
df.groupby('author')['paragraph_neg_wc'].mean()

In [ ]:
grouped = df.groupby(['author', 'category'])['paragraph_neg_wc'].mean()
print(grouped)

In [ ]:
df['paragraph_perplexity_score'] = df['paragraph_perplexity_score'].apply(ast.literal_eval)
df['avg_paragraph_perplexity'] = df['paragraph_perplexity_score'].apply(lambda x: sum(x)/len(x) if x else float('nan'))
grouped = df.groupby(['author', 'category'])['avg_paragraph_perplexity'].mean()

print(grouped)

In [ ]:
df['paragraph_liwc_features'][0]

In [ ]:
df['paragraph_writeprint_eatures'][0]

In [ ]:
from scipy.spatial.distance import jensenshannon
from collections import Counter
import numpy as np

ai_text = "Dog"
human_text = "Cat dog"

ai_tokens = ai_text.lower().replace('.', '').replace('’', "'").split()
human_tokens = human_text.lower().replace('.', '').replace('’', "'").split()

vocab = sorted(set(ai_tokens + human_tokens))

ai_counts = Counter(ai_tokens)
human_counts = Counter(human_tokens)

def normalize(counter, vocab):
    total = sum(counter.values())
    return np.array([counter.get(word, 0) / total for word in vocab])

p = normalize(ai_counts, vocab)
q = normalize(human_counts, vocab)

js_score = jensenshannon(p, q, base=2)

print(f"Jensen-Shannon Divergence (base 2): {js_score:.4f}")


In [ ]:
df_all = pd.read_csv('combined_labels.csv')
df_major = pd.read_csv('fetched_articles.csv')

In [ ]:
grouped = df_all.groupby('Parent_Folder')
df_all.groupby('Parent_Folder')['File'].count()


In [ ]:
import pandas as pd


# Normalize slashes and extract category and year
df_all['Category'] = df_all['Parent_Folder'].apply(lambda x: x.replace('\\', '/').split('/')[-2])
df_all['Year'] = df_all['Parent_Folder'].apply(lambda x: x.replace('\\', '/').split('/')[-1])

# Count number of files per (Year, Category)
file_counts = df_all.groupby(['Year', 'Category'])['File'].count().reset_index()

pivot_table = file_counts.pivot(index='Year', columns='Category', values='File').fillna(0).astype(int)

pivot_table['Total'] = pivot_table.sum(axis=1)

pivot_table.loc['Total'] = pivot_table.sum(axis=0)

print(pivot_table)


In [ ]:
df_major.info()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T  # Eras become index, categories become columns

        # Set seaborn style for better aesthetics
        sns.set_style("whitegrid")
        
        # Create figure with specific size and DPI
        fig, ax = plt.subplots(figsize=(14, 12), dpi=1800)
        
        # Use a professional color palette
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        # Plot bars with enhanced styling
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        # Customize grid
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Set y-axis limit based on whether it's the first (group_1) or second (group_2) plot
        if df_slice.equals(group_1):
            ax.set_ylim(0, 1.0)
        else:
            ax.set_ylim(0, 1.4)
            
        # Enhance text elements
        ax.set_xlabel('', fontsize=26, fontweight='bold', color='black')
        ax.tick_params(axis='x', labelsize=26, width=1.5, colors='black')
        ax.tick_params(axis='y', labelsize=26, width=1.5, colors='black')

        # Bold tick labels
        for label in ax.get_xticklabels():
            label.set_fontweight('bold')
        for label in ax.get_yticklabels():
            label.set_fontweight('bold')

        
        # Move legend to bottom inside the frame with increased font size
        ax.legend(title='Category', fontsize=26, title_fontsize=26, 
                loc='lower right',
                ncol=len(df_rearranged.columns),
                frameon=True,
                edgecolor='black',)

        # Draw connecting rectangles between corresponding bars
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            
            # Get bar coordinates
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            
            # Draw dashed lines forming a rectangle
            plt.plot([x1, x2], [y2, y2], '--', color='black', alpha=0.5, linewidth=2)  # Top horizontal
            plt.plot([x1, x1], [y1, y2], '--', color='black', alpha=0.5, linewidth=2)  # Vertical

        # Annotate bars with percentage
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=26, color='black',
                    weight='bold'
                )

        # Calculate and annotate ratios
        pre_era = 'Pre-GPT Era'
        post_era = 'Post-GPT Era'

        if pre_era in df_rearranged.index and post_era in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]

                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()

                if pre_val > 0:
                    ratio = post_val / pre_val
                    ratio_text = f'{ratio:.2f}x'
                else:
                    ratio_text = '∞'

                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08  # Keep original vertical offset

                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=26, color='darkred',
                    weight='bold'
                )

        # Customize spines
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(1.5)

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    # Group 1
    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')

    # Group 2
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# --- Final plotting calls ---
plot_grouped_figures(table_2, 2, output_folder)
# plot_grouped_figures(table_3, 3, output_folder)  # Uncomment if needed

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T  # Eras become index, categories become columns

        # Set seaborn style for better aesthetics
        sns.set_style("whitegrid")
        
        # Create figure with specific size and DPI
        fig, ax = plt.subplots(figsize=(14, 12), dpi=1800)
        
        # Use a professional color palette
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        # Plot bars with enhanced styling
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        # Customize grid
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Set y-axis limit based on whether it's the first (group_1) or second (group_2) plot
        if df_slice.equals(group_1):
            ax.set_ylim(0, 1.0)
        else:
            ax.set_ylim(0, 1.4)
            
        # Enhance text elements
        ax.set_xlabel('', fontsize=26, fontweight='bold', color='black')
        ax.tick_params(axis='x', labelsize=26, width=1.5, colors='black')
        ax.tick_params(axis='y', labelsize=26, width=1.5, colors='black')

        # Bold tick labels
        for label in ax.get_xticklabels():
            label.set_fontweight('bold')
        for label in ax.get_yticklabels():
            label.set_fontweight('bold')

        
        # Move legend to bottom inside the frame with increased font size
        ax.legend(title='Category', fontsize=26, title_fontsize=26, 
                loc='lower right',
                ncol=len(df_rearranged.columns),
                frameon=True,
                edgecolor='black',)

        # Draw connecting rectangles between corresponding bars
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            
            # Get bar coordinates
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            
            # Draw dashed lines forming a rectangle
            plt.plot([x1, x2], [y2, y2], '--', color='black', alpha=0.5, linewidth=2)  # Top horizontal
            plt.plot([x1, x1], [y1, y2], '--', color='black', alpha=0.5, linewidth=2)  # Vertical

        # Annotate bars with percentage
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=26, color='black',
                    weight='bold'
                )

        # Calculate and annotate ratios
        pre_era = 'Pre-GPT Era'
        post_era = 'Post-GPT Era'

        if pre_era in df_rearranged.index and post_era in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]

                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()

                if pre_val > 0:
                    ratio = post_val / pre_val
                    ratio_text = f'{ratio:.2f}x'
                else:
                    ratio_text = '∞'

                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08  # Keep original vertical offset

                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=26, color='darkred',
                    weight='bold'
                )

        # Customize spines
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(1.5)

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    # Group 1
    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')

    # Group 2
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# --- Final plotting calls ---
plot_grouped_figures(table_2, 2, output_folder)
# plot_grouped_figures(table_3, 3, output_folder)  # Uncomment if needed

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T  # Eras become index, categories become columns

        # Set seaborn style for better aesthetics
        sns.set_style("whitegrid")
        
        # Create figure with specific size and DPI
        fig, ax = plt.subplots(figsize=(14, 12), dpi=1800)
        
        # Use a professional color palette
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        # Plot bars with enhanced styling
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        # Customize grid
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Set y-axis limit based on whether it's the first (group_1) or second (group_2) plot
        if df_slice.equals(group_1):
            ax.set_ylim(0, 1.0)
        else:
            ax.set_ylim(0, 1.4)
            
        # Enhance text elements
        ax.set_xlabel('', fontsize=26, fontweight='bold', color='black')
        ax.tick_params(axis='x', labelsize=26, width=1.5, colors='black')
        ax.tick_params(axis='y', labelsize=26, width=1.5, colors='black')

        # Bold tick labels
        for label in ax.get_xticklabels():
            label.set_fontweight('bold')
        for label in ax.get_yticklabels():
            label.set_fontweight('bold')

        
        # Move legend to bottom inside the frame with increased font size
        ax.legend(title='Category', fontsize=26, title_fontsize=26, 
                loc='lower right',
                ncol=len(df_rearranged.columns),
                frameon=True,
                edgecolor='black',)

        # Draw connecting rectangles between corresponding bars
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            
            # Get bar coordinates
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            
            # Draw dashed lines forming a rectangle
            plt.plot([x1, x2], [y2, y2], '--', color='black', alpha=0.5, linewidth=2)  # Top horizontal
            plt.plot([x1, x1], [y1, y2], '--', color='black', alpha=0.5, linewidth=2)  # Vertical

        # Annotate bars with percentage
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=26, color='black',
                    weight='bold'
                )

        # Calculate and annotate ratios
        pre_era = 'Pre-GPT Era'
        post_era = 'Post-GPT Era'

        if pre_era in df_rearranged.index and post_era in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]

                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()

                if pre_val > 0:
                    ratio = post_val / pre_val
                    ratio_text = f'{ratio:.2f}x'
                else:
                    ratio_text = '∞'

                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08  # Keep original vertical offset

                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=26, color='darkred',
                    weight='bold'
                )

        # Customize spines
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(1.5)

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    # Group 1
    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')

    # Group 2
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# --- Final plotting calls ---
plot_grouped_figures(table_2, 2, output_folder)
# plot_grouped_figures(table_3, 3, output_folder)  # Uncomment if needed

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T  # Eras become index, categories become columns

        # Set seaborn style for better aesthetics
        sns.set_style("whitegrid")
        
        # Create figure with specific size and DPI
        fig, ax = plt.subplots(figsize=(14, 12), dpi=1800)
        
        # Use a professional color palette
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        # Plot bars with enhanced styling
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        # Customize grid
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Set y-axis limit based on whether it's the first (group_1) or second (group_2) plot
        if df_slice.equals(group_1):
            ax.set_ylim(0, 1.0)
        else:
            ax.set_ylim(0, 1.4)
            
        # Enhance text elements
        ax.set_xlabel('', fontsize=26, fontweight='bold', color='black')
        ax.tick_params(axis='x', labelsize=26, width=1.5, colors='black')
        ax.tick_params(axis='y', labelsize=26, width=1.5, colors='black')

        # Bold tick labels
        for label in ax.get_xticklabels():
            label.set_fontweight('bold')
        for label in ax.get_yticklabels():
            label.set_fontweight('bold')

        
        # Move legend to bottom inside the frame with increased font size
        ax.legend(title='Category', fontsize=26, title_fontsize=26, 
                loc='lower right',
                ncol=len(df_rearranged.columns),
                frameon=True,
                edgecolor='black',)

        # Draw connecting rectangles between corresponding bars
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            
            # Get bar coordinates
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            
            # Draw dashed lines forming a rectangle
            plt.plot([x1, x2], [y2, y2], '--', color='black', alpha=0.5, linewidth=2)  # Top horizontal
            plt.plot([x1, x1], [y1, y2], '--', color='black', alpha=0.5, linewidth=2)  # Vertical

        # Annotate bars with percentage
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=26, color='black',
                    weight='bold'
                )

        # Calculate and annotate ratios
        pre_era = 'Pre-GPT Era'
        post_era = 'Post-GPT Era'

        if pre_era in df_rearranged.index and post_era in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]

                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()

                if pre_val > 0:
                    ratio = post_val / pre_val
                    ratio_text = f'{ratio:.2f}x'
                else:
                    ratio_text = '∞'

                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08  # Keep original vertical offset

                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=26, color='darkred',
                    weight='bold'
                )

        # Customize spines
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(1.5)

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    # Group 1
    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')

    # Group 2
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# --- Final plotting calls ---
plot_grouped_figures(table_2, 2, output_folder)
# plot_grouped_figures(table_3, 3, output_folder)  # Uncomment if needed

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T  # Eras become index, categories become columns

        # Set seaborn style for better aesthetics
        sns.set_style("whitegrid")
        
        # Create figure with specific size and DPI
        fig, ax = plt.subplots(figsize=(14, 12), dpi=1800)
        
        # Use a professional color palette
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        # Plot bars with enhanced styling
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        # Customize grid
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Set y-axis limit based on whether it's the first (group_1) or second (group_2) plot
        if df_slice.equals(group_1):
            ax.set_ylim(0, 1.0)
        else:
            ax.set_ylim(0, 1.4)
            
        # Enhance text elements
        ax.set_xlabel('', fontsize=26, fontweight='bold', color='black')
        ax.tick_params(axis='x', labelsize=26, width=1.5, colors='black')
        ax.tick_params(axis='y', labelsize=26, width=1.5, colors='black')

        # Bold tick labels
        for label in ax.get_xticklabels():
            label.set_fontweight('bold')
        for label in ax.get_yticklabels():
            label.set_fontweight('bold')

        
        # Move legend to bottom inside the frame with increased font size
        ax.legend(title='Category', fontsize=26, title_fontsize=26, 
                loc='lower right',
                ncol=len(df_rearranged.columns),
                frameon=True,
                edgecolor='black',)

        # Draw connecting rectangles between corresponding bars
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            
            # Get bar coordinates
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            
            # Draw dashed lines forming a rectangle
            plt.plot([x1, x2], [y2, y2], '--', color='black', alpha=0.5, linewidth=2)  # Top horizontal
            plt.plot([x1, x1], [y1, y2], '--', color='black', alpha=0.5, linewidth=2)  # Vertical

        # Annotate bars with percentage
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=26, color='black',
                    weight='bold'
                )

        # Calculate and annotate ratios
        pre_era = 'Pre-GPT Era'
        post_era = 'Post-GPT Era'

        if pre_era in df_rearranged.index and post_era in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]

                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()

                if pre_val > 0:
                    ratio = post_val / pre_val
                    ratio_text = f'{ratio:.2f}x'
                else:
                    ratio_text = '∞'

                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08  # Keep original vertical offset

                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=26, color='darkred',
                    weight='bold'
                )

        # Customize spines
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(1.5)

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    # Group 1
    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')

    # Group 2
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# --- Final plotting calls ---
plot_grouped_figures(table_2, 2, output_folder)
# plot_grouped_figures(table_3, 3, output_folder)  # Uncomment if needed

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T  # Eras become index, categories become columns

        # Set seaborn style for better aesthetics
        sns.set_style("whitegrid")
        
        # Create figure with specific size and DPI
        fig, ax = plt.subplots(figsize=(14, 12), dpi=1800)
        
        # Use a professional color palette
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        # Plot bars with enhanced styling
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        # Customize grid
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Set y-axis limit based on whether it's the first (group_1) or second (group_2) plot
        if df_slice.equals(group_1):
            ax.set_ylim(0, 1.0)
        else:
            ax.set_ylim(0, 1.4)
            
        # Enhance text elements
        ax.set_xlabel('', fontsize=26, fontweight='bold', color='black')
        ax.tick_params(axis='x', labelsize=26, width=1.5, colors='black')
        ax.tick_params(axis='y', labelsize=26, width=1.5, colors='black')

        # Bold tick labels
        for label in ax.get_xticklabels():
            label.set_fontweight('bold')
        for label in ax.get_yticklabels():
            label.set_fontweight('bold')

        
        # Move legend to bottom inside the frame with increased font size
        ax.legend(title='Category', fontsize=26, title_fontsize=26, 
                loc='lower right',
                ncol=len(df_rearranged.columns),
                frameon=True,
                edgecolor='black',)

        # Draw connecting rectangles between corresponding bars
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            
            # Get bar coordinates
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            
            # Draw dashed lines forming a rectangle
            plt.plot([x1, x2], [y2, y2], '--', color='black', alpha=0.5, linewidth=2)  # Top horizontal
            plt.plot([x1, x1], [y1, y2], '--', color='black', alpha=0.5, linewidth=2)  # Vertical

        # Annotate bars with percentage
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=26, color='black',
                    weight='bold'
                )

        # Calculate and annotate ratios
        pre_era = 'Pre-GPT Era'
        post_era = 'Post-GPT Era'

        if pre_era in df_rearranged.index and post_era in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]

                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()

                if pre_val > 0:
                    ratio = post_val / pre_val
                    ratio_text = f'{ratio:.2f}x'
                else:
                    ratio_text = '∞'

                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08  # Keep original vertical offset

                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=26, color='darkred',
                    weight='bold'
                )

        # Customize spines
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(1.5)

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    # Group 1
    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')

    # Group 2
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# --- Final plotting calls ---
plot_grouped_figures(table_2, 2, output_folder)
# plot_grouped_figures(table_3, 3, output_folder)  # Uncomment if needed

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T  # Eras become index, categories become columns

        # Set seaborn style for better aesthetics
        sns.set_style("whitegrid")
        
        # Create figure with specific size and DPI
        fig, ax = plt.subplots(figsize=(14, 12), dpi=1800)
        
        # Use a professional color palette
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        # Plot bars with enhanced styling
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        # Customize grid
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Set y-axis limit based on whether it's the first (group_1) or second (group_2) plot
        if df_slice.equals(group_1):
            ax.set_ylim(0, 1.0)
        else:
            ax.set_ylim(0, 1.4)
            
        # Enhance text elements
        ax.set_xlabel('', fontsize=26, fontweight='bold', color='black')
        ax.tick_params(axis='x', labelsize=26, width=1.5, colors='black')
        ax.tick_params(axis='y', labelsize=26, width=1.5, colors='black')

        # Bold tick labels
        for label in ax.get_xticklabels():
            label.set_fontweight('bold')
        for label in ax.get_yticklabels():
            label.set_fontweight('bold')

        
        # Move legend to bottom inside the frame with increased font size
        ax.legend(title='Category', fontsize=26, title_fontsize=26, 
                loc='lower right',
                ncol=len(df_rearranged.columns),
                frameon=True,
                edgecolor='black',)

        # Draw connecting rectangles between corresponding bars
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            
            # Get bar coordinates
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            
            # Draw dashed lines forming a rectangle
            plt.plot([x1, x2], [y2, y2], '--', color='black', alpha=0.5, linewidth=2)  # Top horizontal
            plt.plot([x1, x1], [y1, y2], '--', color='black', alpha=0.5, linewidth=2)  # Vertical

        # Annotate bars with percentage
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=26, color='black',
                    weight='bold'
                )

        # Calculate and annotate ratios
        pre_era = 'Pre-GPT Era'
        post_era = 'Post-GPT Era'

        if pre_era in df_rearranged.index and post_era in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]

                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()

                if pre_val > 0:
                    ratio = post_val / pre_val
                    ratio_text = f'{ratio:.2f}x'
                else:
                    ratio_text = '∞'

                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08  # Keep original vertical offset

                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=26, color='darkred',
                    weight='bold'
                )

        # Customize spines
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(1.5)

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    # Group 1
    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')

    # Group 2
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# --- Final plotting calls ---
plot_grouped_figures(table_2, 2, output_folder)
# plot_grouped_figures(table_3, 3, output_folder)  # Uncomment if needed

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T  # Eras become index, categories become columns

        # Set seaborn style for better aesthetics
        sns.set_style("whitegrid")
        
        # Create figure with specific size and DPI
        fig, ax = plt.subplots(figsize=(14, 12), dpi=1800)
        
        # Use a professional color palette
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        # Plot bars with enhanced styling
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        # Customize grid
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Set y-axis limit based on whether it's the first (group_1) or second (group_2) plot
        if df_slice.equals(group_1):
            ax.set_ylim(0, 1.0)
        else:
            ax.set_ylim(0, 1.4)
            
        # Enhance text elements
        ax.set_xlabel('', fontsize=26, fontweight='bold', color='black')
        ax.tick_params(axis='x', labelsize=26, width=1.5, colors='black')
        ax.tick_params(axis='y', labelsize=26, width=1.5, colors='black')

        # Bold tick labels
        for label in ax.get_xticklabels():
            label.set_fontweight('bold')
        for label in ax.get_yticklabels():
            label.set_fontweight('bold')

        
        # Move legend to bottom inside the frame with increased font size
        ax.legend(title='Category', fontsize=26, title_fontsize=26, 
                loc='lower right',
                ncol=len(df_rearranged.columns),
                frameon=True,
                edgecolor='black',)

        # Draw connecting rectangles between corresponding bars
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            
            # Get bar coordinates
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            
            # Draw dashed lines forming a rectangle
            plt.plot([x1, x2], [y2, y2], '--', color='black', alpha=0.5, linewidth=2)  # Top horizontal
            plt.plot([x1, x1], [y1, y2], '--', color='black', alpha=0.5, linewidth=2)  # Vertical

        # Annotate bars with percentage
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=26, color='black',
                    weight='bold'
                )

        # Calculate and annotate ratios
        pre_era = 'Pre-GPT Era'
        post_era = 'Post-GPT Era'

        if pre_era in df_rearranged.index and post_era in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]

                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()

                if pre_val > 0:
                    ratio = post_val / pre_val
                    ratio_text = f'{ratio:.2f}x'
                else:
                    ratio_text = '∞'

                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08  # Keep original vertical offset

                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=26, color='darkred',
                    weight='bold'
                )

        # Customize spines
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(1.5)

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    # Group 1
    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')

    # Group 2
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# --- Final plotting calls ---
plot_grouped_figures(table_2, 2, output_folder)
# plot_grouped_figures(table_3, 3, output_folder)  # Uncomment if needed

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T  # Eras become index, categories become columns

        # Set seaborn style for better aesthetics
        sns.set_style("whitegrid")
        
        # Create figure with specific size and DPI
        fig, ax = plt.subplots(figsize=(14, 12), dpi=1800)
        
        # Use a professional color palette
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        # Plot bars with enhanced styling
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        # Customize grid
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Set y-axis limit based on whether it's the first (group_1) or second (group_2) plot
        if df_slice.equals(group_1):
            ax.set_ylim(0, 1.0)
        else:
            ax.set_ylim(0, 1.4)
            
        # Enhance text elements
        ax.set_xlabel('', fontsize=26, fontweight='bold', color='black')
        ax.tick_params(axis='x', labelsize=26, width=1.5, colors='black')
        ax.tick_params(axis='y', labelsize=26, width=1.5, colors='black')

        # Bold tick labels
        for label in ax.get_xticklabels():
            label.set_fontweight('bold')
        for label in ax.get_yticklabels():
            label.set_fontweight('bold')

        
        # Move legend to bottom inside the frame with increased font size
        ax.legend(title='Category', fontsize=26, title_fontsize=26, 
                loc='lower right',
                ncol=len(df_rearranged.columns),
                frameon=True,
                edgecolor='black',)

        # Draw connecting rectangles between corresponding bars
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            
            # Get bar coordinates
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            
            # Draw dashed lines forming a rectangle
            plt.plot([x1, x2], [y2, y2], '--', color='black', alpha=0.5, linewidth=2)  # Top horizontal
            plt.plot([x1, x1], [y1, y2], '--', color='black', alpha=0.5, linewidth=2)  # Vertical

        # Annotate bars with percentage
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=26, color='black',
                    weight='bold'
                )

        # Calculate and annotate ratios
        pre_era = 'Pre-GPT Era'
        post_era = 'Post-GPT Era'

        if pre_era in df_rearranged.index and post_era in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]

                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()

                if pre_val > 0:
                    ratio = post_val / pre_val
                    ratio_text = f'{ratio:.2f}x'
                else:
                    ratio_text = '∞'

                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08  # Keep original vertical offset

                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=26, color='darkred',
                    weight='bold'
                )

        # Customize spines
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(1.5)

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    # Group 1
    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')

    # Group 2
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# --- Final plotting calls ---
plot_grouped_figures(table_2, 2, output_folder)
# plot_grouped_figures(table_3, 3, output_folder)  # Uncomment if needed

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T

        sns.set_style("whitegrid")

        # Increased figure size
        fig, ax = plt.subplots(figsize=(16, 14), dpi=800)

        # Changed the red color (#d62728) to a different color
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#8E44AD', '#9467bd', '#8c564b']  # Changed red to purple
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        ax.grid(True, linestyle='--', alpha=0.7)
        ax.set_ylim(0, 0.9 if df_slice.equals(group_1) else 1.3)

        # Increased X/Y ticks and labels
        ax.set_xlabel('', fontsize=40, fontweight='bold', color='black')
        # ax.set_ylabel('Percentage', fontsize=32, fontweight='bold', color='black')
        ax.tick_params(axis='x', labelsize=40, colors='black')
        ax.tick_params(axis='y', labelsize=40, colors='black')

        # Further decreased legend size
        ax.legend(
            title='Category',
            fontsize=25,  # Decreased from 26
            title_fontsize=26,  # Decreased from 28
            loc='lower right',
            ncol=len(df_rearranged.columns),
            frameon=True,
            edgecolor='black',
            fancybox=True
        )
        for text in ax.get_legend().get_texts():
            text.set_fontweight('bold')

        # Draw connectors with increased line width
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            plt.plot([x1, x2], [y2, y2], '--', color='black', alpha=0.5, linewidth=3)
            plt.plot([x1, x1], [y1, y2], '--', color='black', alpha=0.5, linewidth=3)

        # Increased bar annotations
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=40, color='black',
                    weight='bold'
                )

        # Increased ratio annotations
        if 'Pre-GPT Era' in df_rearranged.index and 'Post-GPT Era' in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]
                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()
                ratio_text = f'{post_val / pre_val:.2f}x' if pre_val > 0 else '∞'
                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08
                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=39, color='darkred',  # Changed back to darkred
                    weight='bold'
                )

        # Increased spine width
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(2.5)

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# Call the function
plot_grouped_figures(table_2, 2, output_folder)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T  # Eras become index, categories become columns

        colors = plt.get_cmap('Set2').colors

        fig, ax = plt.subplots(figsize=(14, 12))  # Increased figure size
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        ax.set_xlabel('', fontsize=28, labelpad=15)  # Increased x-label font size
        
        # Set y-axis limit based on whether it's the first (group_1) or second (group_2) plot
        if df_slice.equals(group_1):
            ax.set_ylim(0, 1.0)
        else:
            ax.set_ylim(0, 1.4)
            
        ax.tick_params(axis='x', labelsize=22)
        ax.tick_params(axis='y', labelsize=22)
        
        # Move legend to bottom inside the frame with increased font size
        ax.legend(title='Category', fontsize=21, title_fontsize=21, 
                 loc='lower right',
                 ncol=len(df_rearranged.columns))

        # Draw connecting rectangles between corresponding bars
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            
            # Get bar coordinates
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            
            # Draw dashed lines forming a rectangle
            plt.plot([x1, x2], [y2, y2], '--', color='gray', alpha=0.5, linewidth=2)  # Top horizontal
            plt.plot([x1, x1], [y1, y2], '--', color='gray', alpha=0.5, linewidth=2)  # Vertical

        # Annotate bars with percentage
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=25, color='black',
                    weight='bold'
                )

        # Calculate and annotate ratios
        pre_era = 'Pre-GPT Era'
        post_era = 'Post-GPT Era'

        if pre_era in df_rearranged.index and post_era in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]

                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()

                if pre_val > 0:
                    ratio = post_val / pre_val
                    ratio_text = f'{ratio:.2f}x'
                else:
                    ratio_text = '∞'

                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08  # Keep original vertical offset

                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=25, color='darkred',
                    weight='bold'
                )

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=2000)
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    # Group 1
    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')

    # Group 2
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# --- Final plotting calls ---
plot_grouped_figures(table_2, 2, output_folder)
# plot_grouped_figures(table_3, 3, output_folder)  # Uncomment if needed


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Update labels for "Before" and "After"
def relabel_eras(df):
    return df.rename(columns={'Before': 'Pre-GPT Era', 'After': 'Post-GPT Era'})

def plot_grouped_figures(df, n, folder):
    df = relabel_eras(df)

    group_1 = df.loc[df.index.isin(group_1_cats)]
    group_2 = df.loc[~df.index.isin(group_1_cats)]

    def plot_save(df_slice, title_suffix, filename_suffix):
        df_rearranged = df_slice.T  # Eras become index, categories become columns

        colors = plt.get_cmap('Set2').colors

        fig, ax = plt.subplots(figsize=(14, 12))  # Increased figure size
        df_rearranged.plot(kind='bar', ax=ax, rot=0, color=colors[:len(df_rearranged.columns)], width=0.8)

        ax.set_xlabel('', fontsize=28, labelpad=15)  # Increased x-label font size
        
        # Set y-axis limit based on whether it's the first (group_1) or second (group_2) plot
        if df_slice.equals(group_1):
            ax.set_ylim(0, 1.0)
        else:
            ax.set_ylim(0, 1.4)
            
        ax.tick_params(axis='x', labelsize=22)
        ax.tick_params(axis='y', labelsize=22)
        
        # Move legend to best location with increased font size
        ax.legend(title='Category', fontsize=21, title_fontsize=21, 
                 loc='best',
                 ncol=len(df_rearranged.columns))

        # Draw connecting rectangles between corresponding bars
        bars = ax.containers
        for category_bars in bars:
            pre_bar = category_bars[0]
            post_bar = category_bars[1]
            
            # Get bar coordinates
            x1 = pre_bar.get_x() + pre_bar.get_width() / 2
            x2 = post_bar.get_x() + post_bar.get_width() / 2
            y1 = pre_bar.get_height()
            y2 = post_bar.get_height()
            
            # Draw dashed lines forming a rectangle
            plt.plot([x1, x2], [y2, y2], '--', color='gray', alpha=0.5, linewidth=2)  # Top horizontal
            plt.plot([x1, x1], [y1, y2], '--', color='gray', alpha=0.5, linewidth=2)  # Vertical

        # Annotate bars with percentage
        for bar_container in bars:
            for bar in bar_container:
                height = bar.get_height()
                x = bar.get_x() + bar.get_width() / 2
                ax.text(
                    x, height + 0.01,
                    f'{height:.2f}%',
                    ha='center', va='bottom',
                    fontsize=25, color='black',
                    weight='bold'
                )

        # Calculate and annotate ratios
        pre_era = 'Pre-GPT Era'
        post_era = 'Post-GPT Era'

        if pre_era in df_rearranged.index and post_era in df_rearranged.index:
            for category_bars in bars:
                pre_bar = category_bars[0]
                post_bar = category_bars[1]

                pre_val = pre_bar.get_height()
                post_val = post_bar.get_height()

                if pre_val > 0:
                    ratio = post_val / pre_val
                    ratio_text = f'{ratio:.2f}x'
                else:
                    ratio_text = '∞'

                x_pos = pre_bar.get_x() + pre_bar.get_width() / 2
                y_pos = pre_val + 0.08  # Keep original vertical offset

                ax.text(
                    x_pos, y_pos, ratio_text,
                    ha='center', va='bottom',
                    fontsize=25, color='darkred',
                    weight='bold'
                )

        plt.tight_layout()
        filename = os.path.join(folder, f'percentage_ai_flagged_at_least_{n}_{filename_suffix}.png')
        plt.savefig(filename, dpi=2000)
        plt.show()
        plt.close()
        print(f'Saved plot: {filename}')

    # Group 1
    if not group_1.empty:
        plot_save(group_1, 'Major, College, Local (Pre vs Post GPT Era)', 'major_college_local')

    # Group 2
    if not group_2.empty:
        plot_save(group_2, 'Other Categories (Pre vs Post GPT Era)', 'other_categories')

# --- Final plotting calls ---
plot_grouped_figures(table_2, 2, output_folder)
# plot_grouped_figures(table_3, 3, output_folder)  # Uncomment if needed

In [ ]:
import pandas as pd
df = pd.read_csv('Code/Echoes_AI/chess_inspired_human_ai_text_distinction/code/ALL_ANALYSIS.csv')
df